In [0]:
#
# Executing this cell you will install OpenMC and the nuclear
# data libraries in this instance of the Google Colaboratory virtual machine.
# The process takes about 2 minutes.
#

def install_openmc():
  #
  # Clone source code from Github, make and install
  #
  
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  
  %cd -q /content
  print("Obtaining OpenMC source code from Github...")
  !git  --no-pager clone https://github.com/mit-crpg/openmc.git &> /dev/null
  %cd -q openmc
  !git --no-pager checkout master &> /dev/null
  !mkdir build
  %cd -q build
  print("Running cmake...")
  !cmake ../ -DPYTHON_EXECUTABLE=/usr/bin/python3 -DCMAKE_INSTALL_PREFIX=/usr/local &> /dev/null
  print("Running make...")
  !make -j &> /dev/null
  print("Running make install...")
  !make install &> /dev/null
  import sys
  sys.path.append('/usr/local/lib/python3.6/dist-packages/openmc-0.10.0-py3.6-linux-x86_64.egg')
  sys.path.append('/usr/local/lib/python3.6/site-packages/')
  %cd -q /content

def install_data_from_onedrive():
  #
  # Download preprocessed HDF5 files from OneDrive (faster).
  #
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala los datos nucleares de OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  %cd -q /content
  print("Obtaining HDF5 files from OneDrive...")
  !wget -O nndc_hdf5.tar.gz "https://onedrive.live.com/download?cid=22422A8EEA2A85B9&resid=22422A8EEA2A85B9%21105&authkey=AHL6xwYFXDwEzkk" &> /dev/null
  print("Uncompressing...")
  !tar xzvf nndc_hdf5.tar.gz &> /dev/null
    
from time import time
t1 = time()
install_openmc()
install_data_from_onedrive()
t2 = time()
print("Installed OpenMC in {:.2f} minutes".format((t2-t1)/60.0))


Obtaining OpenMC source code from Github...


# Ejemplos de definición de materiales en OpenMC

Comenzamos importando el módulo `openmc`, contiene el API (applicationg programming interface) para generar el input de OpenMC.

In [0]:
import openmc

La definición de materiales en openmc se realiza llamando a la función `Material()`:

In [0]:
agua = openmc.Material()

Una vez definido el material podemos utilizar el método `.add_nuclide()` para agregar nucleidos a la composición:

In [0]:
agua.add_nuclide('H1', 2.0, 'ao')

El primer primer parámetro de `.add_nuclide()` es el nucleido a agregar, en este caso $^1$H, el segundo la fracción de la composición que representa, y el tercer parámetro las unidades de la fracción indicada. `ao` es fracción en cantidad de átomos, y `wo` es fracción en peso. 

Para definir la densidad del material utilizamos el método `.set_density()`:

In [0]:
agua.set_density('g/cm3', 1.0)

Otras unidades posibles para la densidad son átomos por barn*cm (`'atom/b-cm'`) o kg por m$^3$ (`'kg/m3'`).

Se pueden agregar también elementos a la composición con el método `.add_element()`:

In [0]:
agua.add_element('O', 1.0, 'ao')

Al exportar la información en fomato `xml` para ejecutar OpenMC, el API convierte estas composiciones a la distribución isotópica que corresponda a la biblioteca de datos nucleares que se haya especificado ya que algunas bibliotecas poseen datos para los elemenos, otras para todos sus isótopos y otras sólo para algunos. Por ejemplo, si bien el oxígeno natural contiene $^{16}$O, $^{17}$O y $^{18}$O no todas las bibliotecas poseen datos para todos los isótopos, por lo que la composición puede variar.

**Cuando se calculan benchmarks es recomendable especificar en forma explícita la fracción de cada nucleido presente para evitar ambiguedades**

## Datos nucleares térmicos

El tratamiento default para la sección eficaz de scattering para neutrones térmicos es el de gas libre a la temperatura de la celda o material. En caso de que la biblioteca tenga secciones eficaces de scattering térmico, estas pueden especificarse en la definición del material utilizando el método `.add_s_alpha_beta()`:

In [0]:
agua.add_s_alpha_beta("c_H_in_H2O")

Ahora, la definición del material incluirá el reemplazo para energías térmicas de la sección eficaz de scattering del $^{1}$H por la definida en la biblioteca `c_H_in_H2O`.

## Nomenclatura de nucleidos, elementos y compuestos

La nomenclatura utilizada en OpenMC proviene de [GND](https://www.oecd-nea.org/science/wpec/sg38/Meetings/2016_May/tlh4gnd-main.pdf), que es un formato que en el futuro reemplazará a ENDF. En el pasado la especificación de nucleidos se realizaba mediante números enteros (ZAID o MAT). Esto fue reemplazado por strings siguiendo el siguiente criterio:

* Elementos: `Sym0` (el símbolo químico seguido de 0). Por ejemplo, `C0`, `Al0`.
* Nucleidos: `SymA` (el símbolo químico del elemento seguido del número atómico. Por ejemplo, `C12`, `Al27`. 
* Estados excitados: `SymA_eN` (donde N es el número de nivel excitado). Por ejemplo, `V51_e2`.
* Estados metastables: `SymA_mN` (donde N es el número de estado metastable). Por ejemplo, `Am242_m1`.
* Compuestos: `c_nombre` (donde nombre es un string que identifica el compuesto). Por ejemplo, `c_H_in_H2O`.

## Temperaturas

La temperatura de las distintas regiones de la simulación se indica en OpenMC como una propiedad de las celdas. Pero, si uno tiene un material cuya temperatura es única, puede indicarse con el atributo `.temperature`. A diferencia de otros códigos Monte Carlo, la unidad de temperatura es el Kelvin:

In [0]:
agua.temperatura = 293 # K

Si todo el sistema está a una única temperatura, esta puede especificarse en forma global en los parámetros de simulación (settings). La temperatura indicada por celda tiene preeminencia sobre la temperatura de los materiales, y la temperatura de los materiales tiene preeminencia sobre la temperatura fijada en forma global.

## Colección de materiales y generación del input xml

Previo a generar el input para correr es necesario generar una colección de materiales utilizando la función `Materials()`:

In [0]:
mats = openmc.Materials()
mats.append(agua)

o, en forma equivalente:

In [0]:
mats = openmc.Materials([agua])

La generación del input xml se realiza con el método `.export_to_xml()`:

In [0]:
mats.export_to_xml()

Y podemos ver el archivo generado:

In [0]:
!cat materials.xml

In [0]:
agua.get_nuclide_densities()

El objeto de tipo `Materials` tiene una propiedad, `.cross_sections` donde puede indicarse a OpenMC la ubicación de las secciones eficaces.

In [0]:
mats.cross_sections = "/content/nndc_hdf5/cross_sections.xml"

In [0]:
mats.export_to_xml()

In [0]:
!cat materials.xml

Resumiendo, la definición del material agua isótopo por isótopo, es la siguiente:

In [0]:
agua = openmc.Material()
agua.add_nuclide("H1", 2.0*(1.0- 0.0115e-2), 'ao')
agua.add_nuclide("H2", 2.0*0.0115e-2, 'ao')
agua.add_nuclide("O16", 1.0-0.038e-2, 'ao')
agua.add_nuclide("O17", 0.038e-2, 'ao')
agua.add_s_alpha_beta("c_H_in_H2O")
agua.set_density('g/cm3', 1.0)

In [0]:
mats = openmc.Materials()
mats.cross_sections = "/content/nndc_hdf5/cross_sections.xml"
mats.add_material(agua)
mats.export_to_xml()

In [0]:
!cat materials.xml